# ALE explanation using Heart Disease dataset

ALE, like PDP (Partial Dependence Plots) is a model agnostic technique useful for the explaining and visualizing the effects of each feature on the results of black box models. However, it is better compared to PDP as it addresses the primary drawbacks of the same. PD Plots ignore the correlation between the features and considers them to be independent which leads to erroneous results when the features of the dataset are highly correlated. ALE, on the other hand, produce good results in spite of there being a correlation between features and are also less computationally expensive. They visualise the effect that each feature, isolated from all other features, has on the predictions of the model. 

A second order ALE plot can also show the combined effects of multiple features on the outcome. 


In [ ]:
!pip install alibi

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from alibi.explainers.ale import ALE, plot_ale

Reading the Heart disease dataset

In [ ]:
data = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')
# To display the top 5 rows
data.head(5)

In [ ]:
heart = data.copy()

In [ ]:
target = 'condition'
features_list = list(heart.columns)
features_list.remove(target)

In [ ]:
y = heart.pop('condition')

Splitting the data into training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(heart, y, test_size=0.2, random_state=33)

# Using Logistic Regression Model

Fitting a logistic regression model

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

Evaluation the model

In [ ]:
accuracy_score(y_test, lr.predict(X_test))

The model gave a quite good accuracy.

Now we calculate the Accumulated Local Effects using Probability space :

In [ ]:
proba_fun_lr = lr.predict_proba

In [ ]:
proba_ale_lr = ALE(proba_fun_lr, feature_names=features_list, target_names=[0,1])

In [ ]:
proba_exp_lr = proba_ale_lr.explain(X_train.values)

## Probability space

Plotting the ALE plots based on feature effects on the probabilities of each class

In [ ]:
plot_ale(proba_exp_lr, n_cols=3, fig_kw={'figwidth': 12, 'figheight': 15});

The units of Y-axis (ALE) relative probability mass, i.e. given a feature value how much probability the model assigns to each class relative to the mean prediction.


Let us consider cp for example.

In [ ]:
plot_ale(proba_exp_lr, features=[2]);

In this plot,the ALE lines cross the x-axis at approximately 1.6cm, which suggests that for instances of cp around ~1.6cm, the feature effect on the prediction is the same as the average feature effect. Also, going towards the extreme values of the feature, the model assigns a large positive value for 0 and  large negative penalty towards 3 for classification of "0" and vice versa for 1. In other words, the feature effect and the probability is more for instances of higher values of cp when the prediction is 1(having heart disease), and similarly when the cp value is lower, there is more probabolity of no heart disease, as expected.

Now we plot a histogram for cp

In [ ]:
x_train=X_train.to_numpy()
fig, ax = plt.subplots()
for target in range(2):
    ax.hist(x_train[y_train==target][:,2],label=target);

ax.set_xlabel(features_list[2])
ax.legend();

It is clearly seen that as the cp value increases, more no of patients are likely to get the disease.

# 2. Using Gradient Boosting Model :

Now, we see the effects of ALE on a non linear model using Gradient Boosting.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

Testing the accuracy.

In [ ]:
accuracy_score(y_test, gb.predict(X_test))

The accuracy is good enough.


In [ ]:
proba_fun_gb = gb.predict_proba

In [ ]:
proba_ale_gb = ALE(proba_fun_gb, feature_names=features_list, target_names=['0','1'])

In [ ]:
proba_exp_gb = proba_ale_gb.explain(X_train.values[:,:])

In [ ]:
gb.feature_importances_

By printing out the feature importances, it can be deduced that ca and thal are the most important features fbs and restecg are the least important.

Plotting the graphs for all features towards the probability space

In [ ]:
plot_ale(proba_exp_gb, n_cols=2, fig_kw={'figwidth': 15, 'figheight': 15})

Since gradient boosting is applied, the plots are no longer linear.
As seen in the above graphs, fbs and restecg have almost flat lines which clearly indicates that they are the least important features. In contrast, the plot of ca shows a quite increase in feature effect at higher levels of ca.


###Comparing Logistic Regression and Gradient Boosting

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharey='row');
plot_ale(proba_exp_lr, features=[11,2], targets=[1], ax=ax, line_kw={'label': 'LR'});
plot_ale(proba_exp_gb, features=[11,2], targets=[1], ax=ax, line_kw={'label': 'GB'});

We have considered 2 features namely, cp and ca. The following conclusions can be made from the above graphs:

1.   In both models, the feature weight of ca for predicting class '1' is high. It is increasing in nature an has a high positive influence at 3. From about 0 to 0.5 ca, the feature has a high negative influence on predicting the class '1'.
2.   Conversely, the second graph shows that the feature weight of cp has a high importance in the LR Model, but doesn't affect the predictions much on GB Model.

